# TASK I (Text-Only)

## Import important libraries

In [1]:
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras.optimizers import Adam
from keras.models import Sequential, Model
from keras import backend as K
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Input, Conv1D, MaxPooling1D
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np
import pandas as pd
from keras.layers import Input, Dense, Dropout, concatenate, LSTM, Conv2D, MaxPooling2D, Flatten, GlobalMaxPooling1D,  Embedding
from keras.models import Model
from keras.optimizers import Adam, RMSprop
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
from keras.applications.vgg16 import VGG16
from keras.utils import to_categorical
from sklearn.metrics import classification_report, confusion_matrix

In [2]:
import tensorflow as tf
tf.get_logger().setLevel('INFO')

##### Defining some important metrics

In [3]:
def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

## Load the training, testing and valdiation JSONLs

In [4]:
# train_li = []
# # Open the JSONL file
# with open('./train.jsonl', 'r') as f:
#     # Read each line of the file as a separate JSON object
#     for line in f:
#         train_li.append(json.loads(line))
        
# test_li = []
# # Open the JSONL file
# with open('./test_seen.jsonl', 'r') as f:
#     # Read eactest_seen.jsonlh line of the file as a separate JSON object
#     for line in f:
#         test_li.append(json.loads(line))
        
# dev_li = []
# # Open the JSONL file
# with open('./dev_seen.jsonl', 'r') as f:
#     # Read eactest_seen.jsonlh line of the file as a separate JSON object
#     for line in f:
#         dev_li.append(json.loads(line))
        
# ## Getting dataframes
train_path = "C:/Users/Ishit/pytorch/DL_A4/data/df_train_cleaned.csv"
val_path = "C:/Users/Ishit/pytorch/DL_A4/data/df_dev_cleaned.csv"
test_path = "C:/Users/Ishit/pytorch/DL_A4/data/df_train_cleaned.csv"
image_path = "C:/Users/Ishit/Downloads/archive (1)/data/"

train_df = pd.read_csv(train_path)
test_df = pd.read_csv(test_path)
dev_df = pd.read_csv(val_path)

In [5]:
train_text = train_df['text'].tolist()
test_text = test_df['text'].tolist()
dev_text = dev_df['text'].tolist()

In [6]:
y_train = train_df['label'].tolist()
y_test = test_df['label'].tolist()
y_dev = dev_df['label'].tolist()

In [7]:
# unique words
unique_words = set()
for sent in train_text:
    for word in sent.split():
        unique_words.add(word)

print("Number of unique words: ", len(unique_words))

Number of unique words:  8388


### Preprocessing the text

In [8]:
# Preprocess the text data

# max_words = 12000
max_len = 100
tokenizer = Tokenizer()

tokenizer.fit_on_texts(train_df['text'])

sequences_train = tokenizer.texts_to_sequences(train_df['text'])
sequences_val = tokenizer.texts_to_sequences(dev_df['text'])
sequences_test = tokenizer.texts_to_sequences(test_df['text'])

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 6452 unique tokens.


### Getting final train, test, val

In [9]:
x_train = pad_sequences(sequences_train, maxlen=max_len)
x_val = pad_sequences(sequences_val, maxlen=max_len)
x_test = pad_sequences(sequences_test, maxlen=max_len)

y_train = to_categorical(train_df['label'])
y_val = to_categorical(dev_df['label'])
y_test = to_categorical(test_df['label'])

# Preprocess the labels
y_train = np.array(train_df['label'])
y_val = np.array(dev_df['label'])
y_test = np.array(test_df['label'])

In [12]:
from torch.utils.data import Dataset, DataLoader
class TextClassification(Dataset):
    def __init__(self, x, y):
        self.x = x
        self.y = y
        
    def __len__(self):
        return len(self.x)
    
    def __getitem__(self, idx):
        return self.x[idx], self.y[idx]

In [13]:
train_dataset = TextClassification(x_train, y_train)
val_dataset = TextClassification(x_val, y_val)
test_dataset = TextClassification(x_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=True)

In [14]:
import gensim
glove_model = gensim.models.KeyedVectors.load("glove_model")

In [15]:
# implement the model using LSTM in pytorch
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

In [16]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [17]:
class LSTM(nn.Module):
    def __init__(self, embedding_matrix, hidden_size, num_layers, num_classes):
        super(LSTM, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.embedding = nn.Embedding.from_pretrained(embedding_matrix)
        self.lstm = nn.LSTM(embedding_matrix.shape[1], hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)
        
    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        
        out = self.embedding(x)
        out, _ = self.lstm(out, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out

In [18]:
# Hyperparameters
num_epochs = 10
learning_rate = 0.001
hidden_size = 64
num_layers = 2
num_classes = 2

model = LSTM(torch.FloatTensor(glove_model.vectors), hidden_size, num_layers, num_classes).to(device)

In [19]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [20]:
# Train the model
epochs = 10

for epoch in range(epochs):
    train_loss = 0.0
    for i, (data, labels) in enumerate(train_loader):
        data = data.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(data)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        
    print(f'Epoch [{epoch+1}], Loss: {train_loss}')

Epoch [1], Loss: 84.91422459483147
Epoch [2], Loss: 83.66076129674911
Epoch [3], Loss: 82.17684003710747
Epoch [4], Loss: 80.42768996953964
Epoch [5], Loss: 77.32658964395523
Epoch [6], Loss: 73.38861733675003
Epoch [7], Loss: 68.79930329322815
Epoch [8], Loss: 62.21498078107834
Epoch [9], Loss: 55.97354832291603
Epoch [10], Loss: 49.6116042137146


In [11]:
def return_lstm_model_1():
    text_input = Input(shape=(max_len,))

    text_embedding = Embedding(max_words, 128)(text_input)

    text_lstm = LSTM(64)(text_embedding)

    dense1 = Dense(64, activation='relu')(text_lstm)
    dropout1 = Dropout(0.2)(dense1)

    dense2 = Dense(32, activation='relu')(dropout1)
    dropout2 = Dropout(0.2)(dense2)

    dense3 = Dense(8, activation='relu')(dropout2)
    dropout3 = Dropout(0.2)(dense3)

    output = Dense(1, activation='sigmoid')(dropout3)

    model = Model(inputs = text_input, outputs = output)
    
    model.summary()

    model.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.001), metrics=['accuracy', 'Precision', 'Recall', f1_m])
    
    return model

In [12]:
from keras import regularizers

def return_lstm_model_2():
    text_input = Input(shape=(max_len,))

    text_embedding = Embedding(max_words, 128)(text_input)

    text_lstm_1 = LSTM(128, return_sequences=True)(text_embedding)
    text_lstm_2 = LSTM(64)(text_lstm_1)

    dense1 = Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.01))(text_lstm_2)
    dropout1 = Dropout(0.5)(dense1)

    dense2 = Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.01))(dropout1)
    dropout2 = Dropout(0.5)(dense2)

    dense3 = Dense(32, activation='relu', kernel_regularizer=regularizers.l2(0.01))(dropout2)
    dropout3 = Dropout(0.5)(dense3)

    output = Dense(1, activation='sigmoid')(dropout3)

    model = Model(inputs=text_input, outputs=output)
    model.summary()

    model.compile(loss='binary_crossentropy', optimizer=RMSprop(lr=0.001), metrics=['accuracy', 'Precision', 'Recall', f1_m])
    
    return model

In [13]:
def return_lstm_model_3():
    text_input = Input(shape=(max_len,))

    text_embedding = Embedding(max_words, 128)(text_input)

    conv1 = Conv1D(64, 3, activation='relu')(text_embedding)
    pool1 = MaxPooling1D(pool_size=2)(conv1)

    text_lstm = LSTM(64)(pool1)

    dense1 = Dense(64, activation='relu')(text_lstm)
    dropout1 = Dropout(0.2)(dense1)

    dense2 = Dense(32, activation='relu')(dropout1)
    dropout2 = Dropout(0.2)(dense2)

    dense3 = Dense(8, activation='relu')(dropout2)
    dropout3 = Dropout(0.2)(dense3)

    output = Dense(1, activation='sigmoid')(dropout3)

    model = Model(inputs=text_input, outputs=output)
    model.summary()

    model.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.001), metrics=['accuracy', 'Precision', 'Recall', f1_m])

    return model


In [26]:
def return_lstm_model_4():
    text_input = Input(shape=(max_len,))

    # Load pre-trained word embeddings
    embeddings_index = {}
    with open('C:/Users/Ishit/pytorch/DL_A4/data/glove.6B.100d.txt', encoding='utf-8') as f:
        for line in f:
            values = line.split()
            word = values[0]
            coefs = np.asarray(values[1:], dtype='float32')
            embeddings_index[word] = coefs

    # Create embedding matrix
    embedding_dim = 100
    embedding_matrix = np.zeros((max_words, embedding_dim))
    for word, i in word_index.items():
        if i < max_words:
            embedding_vector = embeddings_index.get(word)
            if embedding_vector is not None:
                embedding_matrix[i] = embedding_vector

    # Create embedding layer
    text_embedding = Embedding(max_words, embedding_dim, weights=[embedding_matrix], trainable=False)(text_input)

    text_lstm = LSTM(64)(text_embedding)

    dense1 = Dense(64, activation='relu')(text_lstm)
    dropout1 = Dropout(0.2)(dense1)

    dense2 = Dense(32, activation='relu')(dropout1)
    dropout2 = Dropout(0.2)(dense2)

    dense3 = Dense(8, activation='relu')(dropout2)
    dropout3 = Dropout(0.2)(dense3)
    
    output = Dense(1, activation='sigmoid')(dropout3)

    model = Model(inputs=text_input, outputs=output)
    model.summary()

    model.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.001), metrics=['accuracy', 'Precision', 'Recall', f1_m])

    return model

### Evaluation

In [32]:
def plot_history(history):
    
    # Plot the training and validation loss
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title('Training and Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()

    # Plot the training and validation accuracy
    plt.plot(history.history['accuracy'], label='Training Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.title('Training and Validation Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.show()

In [ ]:
def get_metrics(model):
    
    test_loss, test_acc, test_precision, test_recall, test_f1 = model.evaluate(x_test, y_test)
    train_loss, train_acc, train_precision, train_recall, train_f1 = model.evaluate(x_train, y_train)
    dev_loss, dev_acc, dev_precision, dev_recall, dev_f1 = model.evaluate(x_val, y_val)
    
    
    print("#######################################################################################################")
    print("#######################################################################################################")
    
    print("Following are the TEST metrics associated with the model\n")    
    print(f'Test loss: {test_loss}\n')
    print(f'Test acuuracy: {test_acc}\n')
    print(f'Test precision: {test_precision}\n')
    print(f'Test recall: {test_recall}\n')
    print(f'Test F1-Score: {test_f1}\n')
    y_pred = model.predict(x_test)
    print(classification_report(y_test.reshape(-1,1), y_pred.round()))
    
    print("#######################################################################################################")
    print("#######################################################################################################")
    print("Following are the TRAIN metrics associated with the model\n")    
    print(f'Train loss: {train_loss}\n')
    print(f'Train acuuracy: {train_acc}\n')
    print(f'Train precision: {train_precision}\n')
    print(f'Train recall: {train_recall}\n')
    print(f'Train F1-Score: {train_f1}\n')
    y_pred_train = model.predict(x_train)
    print(classification_report(y_train.reshape(-1,1), y_pred_train.round()))
    
    print("#######################################################################################################")
    print("#######################################################################################################")
    print("Following are the DEV metrics associated with the model\n")    
    print(f'Train loss: {dev_loss}\n')
    print(f'Train acuuracy: {dev_acc}\n')
    print(f'Train precision: {dev_precision}\n')
    print(f'Train recall: {dev_recall}\n')
    print(f'Train F1-Score: {dev_f1}\n')
    y_pred_dev = model.predict(x_val)
    print(classification_report(y_val.reshape(-1,1), y_pred_dev.round()))

### Model 4

In [40]:
# lstm_model_4 = return_lstm_model_4()
# history_lstm_4 = lstm_model_4.fit(x_train, y_train, epochs=50, batch_size=32, validation_data=(x_val, y_val))

In [41]:
# get_metrics(lstm_model_4)

In [42]:
# plot_history(history_lstm_4)

In [ ]:
class 